In [1]:
import yaml
import os
from pprint import pprint
from SmartAITool.core import bprint
# Load API configuration from a YAML file
with open("/home/rteam2/m15kh/LLM/config.yaml", "r") as file:
    config = yaml.safe_load(file)

os.environ["LANGSMITH_TRACING"] = config.get("LANGSMITH_TRACING", "false")
os.environ["LANGSMITH_API_KEY"] = config["LANGSMITH_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ['HUGGINGFACEHUB_API_TOKEN'] = config['HUGGINGFACEHUB_API_TOKEN']
GILAS_API_KEY = config['GILAS_API_KEY']

KeyError: 'GILAS_API_KEY'

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 2)
question = 'who is the currently  president of iran'
search_results = search.invoke(question)
print(search_results)
tools = [search]


[{'title': 'President of Iran - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/President_of_Iran', 'content': 'The president of the Islamic Republic of Iran (Persian: رئیس\u200cجمهور ایران, romanized:\xa0Rais Jomhur-e Irān) is the head of government of the Islamic Republic of Iran and the second highest-ranking official, after the supreme leader.[3] The first election was held in 1980 and was won by Abulhassan Banisadr. Masoud Pezeshkian currently serves as the president of Iran, after being elected in the 2024 Iranian presidential election and being officially endorsed by the supreme leader.[4] [...] | This articleneeds additional citations forverification.Please helpimprove this articlebyadding citations to reliable sources. Unsourced material may be challenged and removed.Find sources:"President of Iran"–news·newspapers·books·scholar·JSTOR(April 2025)(Learn how and when to remove this message)\nPresident of theIslamic Republic of Iran\nرئیس جمهوری اسلامی ایران(Persian)\nSeal of t

In [3]:
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model

model = init_chat_model(
    api_key= GILAS_API_KEY,
    base_url="https://api.gilas.io/v1/",
    model="gpt-4o")



In [6]:
from langchain_core.messages import HumanMessage

model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

pprint(f"ContentString: {response.content}")
pprint(f"ToolCalls: {response.tool_calls}")

'ContentString: Hello! How can I assist you today?'
'ToolCalls: []'


In [7]:
response = model_with_tools.invoke([HumanMessage(content=question)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current president of Iran 2023'}, 'id': 'call_52a8xMi50jQRpSpI00m9M3WY', 'type': 'tool_call'}]


In [15]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content=question)]}
)
response["messages"]

[HumanMessage(content='who is the currently  president of iran', additional_kwargs={}, response_metadata={}, id='d25eb450-c9aa-45ef-8ab6-098d6dca3950'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hQLIJsAKeXi0dh2xHH1WWPBl', 'function': {'arguments': '{"query":"current president of Iran 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 87, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUB6hZ8r4gcpX36ymBM4iu9JJ3ZJw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f7520b4f-17a4-46f5-a741-469ba42c9b8d-0', tool_calls=[{'name': 'tavily_search_result

In [16]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

who is the currently  president of iran
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_pBFC7zK9oTPpccjM5Rg1vOF0)
 Call ID: call_pBFC7zK9oTPpccjM5Rg1vOF0
  Args:
    query: current president of Iran 2023
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "List of presidents of Iran - Wikipedia", "url": "https://en.wikipedia.org/wiki/List_of_presidents_of_Iran", "content": "Ali Khamenei, Akbar Hashemi Rafsanjani, Mohammad Khatami, Mahmoud Ahmadinejad and Hassan Rouhani were each elected president for two terms. Ebrahim Raisi was the eighth president, serving from his election in 2021 until his death on May 19, 2024 leaving the office occupied in an acting capacity by First Vice President Mohammad Mokhber. Masoud Pezeshkian was elected as the current pres

In [17]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [18]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 83, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUBBChe7Oorr8HeyQFsSVyzE3fWyS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9660b67f-3468-47ff-9066-e22755934043-0', usage_metadata={'input_tokens': 83, 'output_tokens': 12, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 106, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUBBcph9OpWNqvThCi7cWdOaEDLyE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--becbe6c7-e50e-494e-8593-a6c149d5e5fa-0', usage_metadata={'input_tokens': 106, 'output_tokens': 7, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [21]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm sorry, but I don't have access to your personal information, including your name.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 84, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUBC4dq6WNVe9mvGDD4COlZIRvMBX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0a0c31c-ac76-42d9-ad13-3c2796e03e09-0', usage_metadata={'input_tokens': 84, 'output_tokens': 19, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
